In [1]:
import ftplib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)
paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [3]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2023-09-04
04092023
VentaCARE_ASSISTANCE_04092023.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_04092023.xlsx


In [4]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [5]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [6]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [7]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape


c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2414, 17)

In [8]:
archivoBCI.dtypes


POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [9]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,16368915,4,FEMENINO,TITULAR MASCULINO,TANIA REYES SANCHEZ,1987-05-05,tania.reyes.s1987@gmail.com,SN,950999267
1,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,27877512,7,FEMENINO,HIJO,FRANCISCO ALONSO PERES REYES,2022-08-02,,SN,0
2,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,25837523,8,FEMENINO,HIJA,JULIETA FERNANDA PERES REYES,2017-07-18,,SN,0
3,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,23407616,7,FEMENINO,HIJA,RENATA MAGDALENA PERES REYES,2010-08-28,,SN,0
4,2602260,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,11164052,1,11164052,1,MASCULINO,TITULAR MASCULINO,NESTOR VIDAL BASCUNAN,1967-03-07,vidal.nestor67@gmail.com,PEDRO MONTT 980,982339878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,2614309,2023-07-01,2024-06-30,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,27206756,2,27206756,2,MASCULINO,TITULAR MASCULINO,ABRAHAM JOSUE OLLARVEZ FRANCO,1992-03-14,abrahamollarves@gmail.com,PASAJE MANUEL OLGUIN TAPIA 2021,946346203
2360,2614309,2023-07-01,2024-06-30,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,27206756,2,27190877,6,MASCULINO,ESPOSA,YAMILA SALAS FLORES,1993-12-29,,SANTIAGO,0
2361,2615394,2023-07-01,2024-06-30,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,15067817,K,23669045,8,MASCULINO,HIJO,LEN EMILIANO JOS FERNANDO DANIEL VSQUEZ ARANCIBIA,2011-06-10,NO INFORMADO,NO INFORMADO,0
2362,2615432,2023-07-02,2024-07-01,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16832354,9,16832354,9,MASCULINO,TITULAR MASCULINO,GABRIEL VICENTE KAUAK KUSCHEL,1988-09-05,GKAUAK@GMAIL.COM,NO INFORMADO,0


In [10]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [11]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_15336\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [12]:
archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,16368915,4,...,950999267,163689154,163689154,2602258,Mujer,tania.reyes.s1987@gmail.com,,950999267,Tania Reyes,Reyes Sanchez
1,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,27877512,7,...,,163689154,278775127,2602258,Mujer,,,,Francisco Alonso,Peres Reyes
2,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,25837523,8,...,,163689154,258375238,2602258,Mujer,,,,Julieta Fernanda,Peres Reyes
3,2602258,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,16368915,4,23407616,7,...,,163689154,234076167,2602258,Mujer,,,,Renata Magdalena,Peres Reyes
4,2602260,2023-06-01,2024-05-31,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,11164052,1,11164052,1,...,982339878,111640521,111640521,2602260,Hombre,vidal.nestor67@gmail.com,Pedro Montt 980,982339878,Nestor Vidal,Vidal Bascunan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,2614309,2023-07-01,2024-06-30,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,27206756,2,27206756,2,...,946346203,272067562,272067562,2614309,Hombre,abrahamollarves@gmail.com,Pasaje Manuel Olguin Tapia 2021,946346203,Abraham Josue,Ollarvez Franco
2360,2614309,2023-07-01,2024-06-30,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,27206756,2,27190877,6,...,,272067562,271908776,2614309,Hombre,,Santiago,,Yamila Salas,Salas Flores
2361,2615394,2023-07-01,2024-06-30,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,15067817,K,23669045,8,...,,15067817K,236690458,2615394,Hombre,,,,Len Emiliano,Vsquez Arancibia
2362,2615432,2023-07-02,2024-07-01,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16832354,9,16832354,9,...,,168323549,168323549,2615432,Hombre,gkauak@gmail.com,,,Gabriel Vicente,Kauak Kuschel


In [13]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [48]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value
        
""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """

In [15]:
results = sf.query_all(
    """
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Canal Individual'
   """
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["rutes"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["rutes"] = pacienteProgr["rutes"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["rutes",]]

""" Pacientes_SF.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
) """
pacienteProgr.shape


(1206, 1)

In [16]:
pacienteProgr["rutes"] = pacienteProgr["rutes"].astype(str).str.upper()
pacienteProgr["rutes"] = pacienteProgr["rutes"].str.replace(" ", "")
pacienteProgr["rutes"] = pacienteProgr["rutes"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(pacienteProgr["rutes"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]

archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI.drop_duplicates(["Rut Carga", "Rut Titular"], keep="last")

archivoBCI = archivoBCI[["Rut Carga","sexo","direccion","email","celular","nombres","apellidos"]]

archivoBCI = archivoBCI.rename(
    columns={"Rut Carga": "IdentificationId__pc", "sexo": "HealthCloudGA__Gender__pc", "direccion":"BillingStreet", "email":"PersonEmail","celular":"Phone","nombres":"FirstName","apellidos":"LastName"}
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(archivoBCI["PersonEmail"]!= "" , "True", "False")
archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
)
derty = sf.quick_search("Jones")

archivoBCI.columns

PermissionError: [Errno 13] Permission denied: 'F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx'

In [53]:
derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)





searchRecords -> [OrderedDict([('attributes', OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])), ('Id', '0018c00002anTmeAAE')]), OrderedDict([('attributes', OrderedDict([('type', 'User'), ('url', '/services/data/v42.0/sobjects/User/0058c00000DSxZmAAL')])), ('Id', '0058c00000DSxZmAAL')]), OrderedDict([('attributes', OrderedDict([('type', 'Contact'), ('url', '/services/data/v42.0/sobjects/Contact/0038c000036iZ1TAAU')])), ('Id', '0038c000036iZ1TAAU')]), OrderedDict([('attributes', OrderedDict([('type', 'Task'), ('url', '/services/data/v42.0/sobjects/Task/00T8c00004uc71MEAQ')])), ('Id', '00T8c00004uc71MEAQ')]), OrderedDict([('attributes', OrderedDict([('type', 'Task'), ('url', '/services/data/v42.0/sobjects/Task/00T8c00004uc70sEAA')])), ('Id', '00T8c00004uc70sEAA')]), OrderedDict([('attributes', OrderedDict([('type', 'Task'), ('url', '/services/data/v42.0/sobjects/Task/00T8c00004uc6PMEAY')])), ('Id', '00T8c00004uc6PMEAY')]), OrderedDic

In [ ]:
mdapi = sf.mdapi
custom_object = mdapi.CustomObject(
    fullName = "CustomObject__c",
    label = "Custom Object",
    pluralLabel = "Custom Objects",
    nameField = mdapi.CustomField(
        label = "Name",
        type = mdapi.FieldType("Text")
    ),
    deploymentStatus = mdapi.DeploymentStatus("Deployed"),
    sharingModel = mdapi.SharingModel("Read")
)